# The Guessing Game


Get your API key by signing up at "https://console.groq.com/keys" and save it in **.env** file as **GROQ_API_KEY**="Your API key"

In [54]:
from dotenv import load_dotenv

load_dotenv() 

True

In [48]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


class Player:
    def __init__(self, model):
        self.observations = []
        self.model = model
        self.concept = None
        self.history = []

    def initialize_host(self):
        template = """
        You are the host of a game where a player asks questions about
        a thing to guess what it is.

        Write the name of a thing. It must be a common object.
        It must be a single word. Do not write anything else. 
        Only write the name of the thing with no punctuation.

        Here is a list of things you cannot use (Make sure not to use these):
        {history}
        """
        prompt = PromptTemplate.from_template(template)
        chain = prompt | self.model | StrOutputParser()

        self.concept = chain.invoke({"history": "\n".join(self.history)})
        self.history.append(self.concept)

        print(f"Concept: {self.concept}")

    def initialize_player(self):
        self.observations = []

    def ask(self, questions_left):

        template = """
        You are a player in a game where you must ask Yes/No questions to identify a common object, 
        which is a single word.

        Here are the questions you've already asked:

        {observations}

        You have {questions_left} questions left. To maximize your chances, 
        aim to guess the object with as few questions as possible. 
        If only 1 question remains, you must make a guess or lose the game. 
        Use past questions to guide your final guess.

        Be aggressive in your approach. Start with broad categories like whether the 
        object is found indoors or outdoors. If it's indoors, narrow down further 
        by considering rooms like the kitchen, living room, or bedroom. 
        Then, focus on specific items within those categories. Try answer as 
        soon as possible. 

        Be as concise as possible when asking a question. Do not anounce that you
        will ask the question. Do not say "Let's get started", or introduce your 
        question. Just write the question.


        Make any guess based on response to your questions if you have not made
        any guess if its your last question. Guess is a thing actually. Is it not
        a category. Thing is pencil, pen, or things like that, final guess is not
        is a used in room or kitchen or office etc.

        Remember:

        Ask only binary questions.
        Be concise; avoid unnecessary introductions to your questions.

        Examples of good questions:
        Is it a fruit?
        Is it bigger than a car?
        Is it alive?

        Examples of bad questions:
        Can I ask a question?
        Can you tell me more about the thing?
        What is the thing?
        How does the thing look?

        """

        prompt = PromptTemplate.from_template(template)
        chain = prompt | self.model | StrOutputParser()
        return chain.invoke(
            {
                "observations": "\n".join(self.observations),
                "questions_left": questions_left,
            }
        )

    def answer(self, question):
        template = """
        You are the host of a game where a player asks questions about
        a {concept} trying to guess what it is.

        The player has asked you the following question: {question}.

        If the player guessed that the thing is "{concept}", answer with
        the word "GUESSED". If the question refers to "{concept}", answer
        with the word "GUESSED".

        If the player didn't guessed, answer the question with a 
        simple Yes or No. Do not say anything else. Do not use any
        punctuation.
        """
        prompt = PromptTemplate.from_template(template)
        chain = prompt | self.model | StrOutputParser()
        return chain.invoke({"concept": self.concept, "question": question})

    def add_observation(self, question, answer):
        self.observations.append(f"Question: {question}. Answer: {answer}")

In [49]:
#from langchain_community.llms import Ollama

from langchain_groq import ChatGroq

player = Player(model=ChatGroq(model="Llama3-70b-8192"))
player.initialize_host()

Concept: Pencil


In [50]:
question = player.ask(20)
question

'Is it something you can hold in your hand?'

In [51]:
player.answer(question)

'Yes'

In [52]:
class Game:
    def __init__(self, model1, model2, rounds=3, questions=20):
        self.model1 = model1
        self.model2 = model2
        self.rounds = rounds
        self.questions = questions

    def start(self):
        players = {
            "0": {
                "player": Player(model=self.model1),
                "score": 0,
            },
            "1": {
                "player": Player(model=self.model2),
                "score": 0,
            },
        }

        host_index = 0
        for round in range(self.rounds):
            print(f"\nRound {round + 1}. Player {host_index + 1} is the host.")

            player_index = 1 - host_index
            if self._play(
                players[str(host_index)]["player"], players[str(player_index)]["player"]
            ):
                print(f"Player {player_index + 1} guessed correctly.")
                players[str(player_index)]["score"] += 1
            else:
                print(f"Player {player_index + 1} didn't guess correctly.")
                players[str(host_index)]["score"] += 1

            host_index = 1 - host_index

        print("Final score:")
        print(f"Player 1: {players['0']['score']}")
        print(f"Player 2: {players['1']['score']}")

    def _play(self, host, player):
        host.initialize_host()
        player.initialize_player()
        for question_index in range(self.questions):
            question = player.ask(self.questions - question_index)
            answer = host.answer(question)

            print(f"Question {question_index + 1}: {question}. Answer: {answer}")

            player.add_observation(question, answer)

            if "guessed" in answer.lower():
                return True

        return False

In [53]:
from langchain_groq import ChatGroq

game = Game(
    model1=ChatGroq(model="Llama3-70b-8192"),
    model2=ChatGroq(model_name="Llama3-70b-8192"),
    rounds=5,
)
game.start()


Round 1. Player 1 is the host.
Concept: Pencil
Question 1: Is it something that can be held in your hand?. Answer: Yes
Question 2: Is it used indoors?. Answer: Yes
Question 3: Is it used for personal grooming?. Answer: No
Question 4: Is it found in a kitchen?. Answer: No
Question 5: Is it used for entertainment?. Answer: No
Question 6: Is it made of plastic?. Answer: No
Question 7: Is it made of fabric?. Answer: No
Question 8: Is it made of metal?. Answer: No
Question 9: Is it used for a specific task?. Answer: Yes
Question 10: Is it used primarily for a functional purpose?. Answer: Yes
Question 11: Is it used primarily with one hand?. Answer: Yes
Question 12: Is it used to manipulate or handle something else?. Answer: Yes
Question 13: Is it used to manipulate or handle food or drink?. Answer: No
Question 14: Is it found in a bedroom?. Answer: No
Question 15: Is it used in an office?. Answer: Yes
Question 16: Is it used to write or draw?. Answer: Yes
Question 17: Is it something that 